<a href="https://colab.research.google.com/github/affanahmed373/Urdu_OCR/blob/main/Urdu_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


from tensorflow.keras import datasets, layers, models, Sequential
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

Data Augmentation function

In [ ]:
def add_gaussian_noise(image):
    # image must be scaled in [0, 1]
    with tf.name_scope('Add_gaussian_noise'):
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=(50)/(255), dtype=tf.float32)
        noise_img = image + noise
        noise_img = tf.clip_by_value(noise_img, 0.0, 1.0)
    return noise_img

from scipy.signal import convolve2d
def GradientMagnitude(image):

    # 4. Gradient Magnitude
    kernerl1 = np.array([
        [-1,-1,-1],
        [0,0,0],
        [1,1,1]
    ])
    kernerl2 = np.array([
        [-1,0,1],
        [-1,0,1],
        [-1,0,1]
    ])
    
    # temp1 = convolve2d(image,kernerl1,mode='same')
    # temp2 = convolve2d(image,kernerl2,mode='same')
    
    temp1 = convolve2d(image,kernerl1,mode='same')
    temp2 = convolve2d(image,kernerl2,mode='same')
    
    temp3 = np.sqrt(temp1**2 + temp2**2)
    
    plt.axis('off')
    plt.imshow(temp3,cmap='gray')
    plt.show()

    return temp3

def GradientDirection(image):

    # 5. Gradient Direction
    kernerl1 = np.array([
        [-1,-1,-1],
        [0,0,0],
        [1,1,1]
    ])
    kernerl2 = np.array([
        [-1,0,1],
        [-1,0,1],
        [-1,0,1]
    ])
    
    temp1 = convolve2d(image,kernerl1,mode='same')
    temp2 = convolve2d(image,kernerl2,mode='same')
    
    temp3 = np.arctan(temp1/temp2)

    plt.axis('off')
    plt.imshow(temp3,cmap='gray')
    plt.show()

    return temp3

def SobelGradientMagnitude(image):

    # 6. Sobel Gradient Magnitude
    kernerl1 = np.array([
        [1,2,1],
        [0,0,0],
        [-1,-2,-1]
    ])
    kernerl2 = np.array([
        [1,0,-1],
        [2,0,-2],
        [1,0,-1]
    ])
    
    temp1 = convolve2d(image,kernerl1,mode='same')
    temp2 = convolve2d(image,kernerl2,mode='same')
    
    temp3 = np.sqrt(temp1**2 + temp2**2)
    
    # plt.axis('off')
    # plt.imshow(temp3,cmap='gray')
    # plt.show()
    
    return temp3

def SobelGradientDirection(image):

    # 7. Sobel Gradient Direction
    kernerl1 = np.array([
        [1,2,1],
        [0,0,0],
        [-1,-2,-1]
    ])
    kernerl2 = np.array([
        [1,0,-1],
        [2,0,-2],
        [1,0,-1]
    ])
    
    temp1 = convolve2d(image,kernerl1,mode='same')
    temp2 = convolve2d(image,kernerl2,mode='same')
    
    temp3 = np.arctan(temp1/temp2)
    
    plt.axis('off')
    plt.imshow(temp3,cmap='gray')
    plt.show()
    
    return temp3

def GuassianBlur(image):

    # 8. Guassian Blur
    import scipy 
    
    temp = scipy.ndimage.filters.gaussian_filter(
        image,
        sigma = 10
        )
    plt.axis('off')
    plt.imshow(temp,cmap='gray')
    plt.show()
    
    return temp

def Sharpening(image):

    # 9. Sharpening
    kernerl1 = np.array([
        [1,1,1],
        [0,0,0],
        [-1,-1,-1]
    ])
    kernerl2 = np.array([
        [1,0,-1],
        [1,0,-1],
        [1,0,-1]
    ])
    
    temp1 = convolve2d(image,kernerl1,mode='same')
    temp2 = convolve2d(image,kernerl2,mode='same')
    
    temp3 = np.sqrt(temp1**2 + temp2**2)
    
    plt.axis('off')
    plt.imshow(temp3 +image ,cmap='gray')
    plt.show()
    
    return temp3+image
    
def Emboss(image, param):
    
    # 10. Emboss
    kernerl = np.array([
        [-1,-1,0],
        [-1,0,1],
        [0,1,1]
    ])
    
    temp = convolve2d(image,kernerl,mode='same') + param
    plt.axis('off')
    plt.imshow(temp,cmap='gray')
    plt.show()
    
    return temp

In [ ]:
DATADIR = "/content/drive/MyDrive/dataset1/ocrdata"
FOLDERS = ["0","1", "2","3","4","5","6","7","8","9"]
CATEGORIES = ["0","1", "2","3","4","5","6","7","8","9"]
NUM_CLASSES = 10
IMG_SIZE = 32


In [ ]:
def read_data():
    images = []
    labels = []
    img_paths = []
    data = []

    for category,folder in zip(CATEGORIES,FOLDERS):  # do B and M 
        path = os.path.join(DATADIR,folder)  # create path to B and M  
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=B and 1=M
        
        for i,img_path in enumerate(tqdm(os.listdir(path))):  # iterate over each image 
            try:
                img_array = cv2.imread(os.path.join(path,img_path))  # convert to array
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size                
                img_paths.append(os.path.join(path,img_path))
                
                data.append([img_array, class_num])  # add this to our training_data
                images.append(img_array)
                labels.append(class_num)
            except Exception as e:  # in the interest in keeping the output clean...
                pass

    images = np.array(images)
    images = np.asarray(images).astype(np.float32)
    labels = np.array(labels)
    

    return images,labels,img_paths 
    
images,labels,img_paths = read_data()


FileNotFoundError: ignored

In [ ]:
'''test_images=[]
train_images=[]
test_labels=[]
train_labels=[]
index = []
j=0
for i in range (120):
 
 
  if (j==10)|(j==11):
    
    test_images.append(images[i])
    test_labels.append(labels[i])
    index.append(i)
  else:
    
    train_images.append(images[i])
    train_labels.append(labels[i])

  if j==11:
    j=0
  else:
    j=j+1
'''

In [ ]:
"""
print (index)
train_labels = np.array(train_labels)
train_images = np.array(train_images)
test_labels = np.array(test_labels)
test_images = np.array(test_images)

print(type(test_images))
print(type(test_labels))
print(type(train_images))
print(type(train_labels))
print (np.size(train_images))
print (np.size(train_labels))
print (np.size(test_images))
print (np.size(test_labels))
"""

In [ ]:
"""
test_images = test_images/255
train_images = train_images/255
X = images
"""

In [ ]:
class_names = ["0","1", "2","3","4","5","6","7","8","9"]
images =images/255
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i*12])
    plt.xlabel(class_names[labels[i*12]])
plt.show()

In [ ]:
model = models.Sequential()
#i used add data augmentation instead of augmenting dataset we do it in runtime....or add in sequential(........)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
def tf_cross_val_score(model, X, y, n_classes):
    """
    Takes features, number of classes, labels and feature vector names and returns validation scores.
    
    X Numpy ndarray - features
    y Numpy ndarray - labels
    n_classes int - number of classes
    """
    histories = []
    scores = []
    acc_per_fold = []
    loss_per_fold = []
    fold_no = 10
    kf = KFold(n_splits = 2, random_state = None, shuffle = False)
    for train_index, test_index in kf.split(X):
        #print(len(train_index))
        X_train,y_train = np.array(X[train_index]),np.array(y[train_index])
        X_test,y_test = np.array(X[test_index]),np.array(y[test_index])
        y_train = tf.one_hot(y_train, NUM_CLASSES)
        #print(len(y_train))
        y_test = tf.one_hot(y_test, NUM_CLASSES)
        
        histres = model.fit(X_train,y_train, epochs=epochs, verbose=2)
        import gc
        gc.collect()

        histories.append(histres)
        score = model.evaluate(X_test,y_test, verbose=0)
        scores.append(score)

        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]};')
        fold_no += 1
    return histories, scores

In [ ]:
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

"""
history = model.fit(train_images, train_labels, epochs= 20, batch_size=10,
                    validation_data=(test_images, test_labels), verbose=2)
"""
histories,scores = tf_cross_val_score(model, X=images, y=labels, epochs= 20, n_classes=NUM_CLASSES)

In [ ]:
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label = 'Test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
for i in range (0,10):
  img_path = f'/content/drive/MyDrive/dataset1/predict/{i}.PNG'
#  print('/content/drive/MyDrive/dataset1/predict/'{i}'.PNG')

  img = keras.preprocessing.image.load_img(
    img_path, target_size=(IMG_SIZE, IMG_SIZE)
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  print(
    "This image {} most likely belongs to {} with a {:.2f} percent confidence."
    .format(i, class_names[np.argmax(score)], 100 * np.max(score))
  )

In [ ]:
!mkdir -p saved_model
model.save('saved_model/urduocr')